In [1]:
import numpy as np
import pandas as pd
import json
import re

In [2]:
# load json as dict
with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_1.json') as json_file:
    dict_1 = json.load(json_file)

with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_2.json') as json_file:
    dict_2 = json.load(json_file)

with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_3.json') as json_file:
    dict_3 = json.load(json_file)

with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_4.json') as json_file:
    dict_4 = json.load(json_file)

all_records = dict_1 + dict_2 + dict_3 + dict_4

# delete some key
for i in range(len(all_records)):
    del all_records[i]['url'], all_records[i]['title'], all_records[i]['description'], all_records[i]['images']

In [14]:
def split_number(data):
    try:
        return re.findall('[(\d+(\.\d+)?)|(\.?\d+)]+', data)[0]
    except:
        return np.nan

def extract_district(data):
    districts = ['Ba Đình', 'Hoàn Kiếm', 'Tây Hồ', 'Long Biên', 'Cầu Giấy', 'Đống Đa', 'Hai Bà Trưng', 'Hoàng Mai', 
                 'Thanh Xuân', 'Sóc Sơn', 'Đông Anh', 'Gia Lâm', 'Nam Từ Liêm', 'Thanh Trì', 'Bắc Từ Liêm', 'Mê Linh', 
                 'Hà Đông', 'Sơn Tây', 'Ba Vì', 'Phúc Thọ', 'Đan Phượng', 'Hoài Đức', 'Quốc Oai', 'Thạch Thất', 
                 'Chương Mỹ', 'Thanh Oai', 'Thường Tín', 'Phú Xuyên', 'Ứng Hòa', 'Mỹ Đức']
    
    for district in districts:
        if district.lower() in data:
            return district
    return np.nan

def extract_million(data):
    if 'triệu' in data:
        try:
            return float(re.findall('[(\d+(\.\d+)?)|(\.?\d+)]+', data)[0])
        except:
            return 0
    return 0

def extract_billion(data):
    if 'tỷ' in data:
        try:
            return float(re.findall('[(\d+(\.\d+)?)|(\.?\d+)]+', data)[0])*1000
        except:
            return 0
    return 0

def extract_price(df):
    df['million'] = df['price'].apply(extract_million).astype(np.float64)
    df['billion'] = df['price'].apply(extract_billion).astype(np.float64)
    df['billion_to_million'] = df['billion'] / df['acreage']
    
    df['price'] = df['billion_to_million'] + df['million']
    df['price'] = df['price'].replace(0, np.nan)
    
    return df.drop(['million', 'billion', 'billion_to_million'], axis=1)

    
def process_data(df):
    # extract bedroom, toilet, acreage to number values
    df['bedroom'] = df['bedroom'].apply(split_number).astype(np.float64)
    df['toilet'] = df['toilet'].apply(split_number).astype(np.float64)
    df['acreage'] = df['acreage'].apply(split_number).astype(np.float64)

    # extract address to district values
    df['address'] = df['address'].apply(extract_district)
    
    # drop nan value of columns address, acreage, bedroom
    df = df.dropna(subset=['address', 'acreage', 'bedroom'])
    
    # split price million and billion
    df = extract_price(df)
    
    return df

In [17]:
# dicts to dataframe and replace none to np.nan
df = pd.DataFrame(all_records).fillna(value=np.nan)
df = process_data(df)

<ipython-input-14-cf1588ec92cd>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['million'] = df['price'].apply(extract_million).astype(np.float64)
<ipython-input-14-cf1588ec92cd>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['billion'] = df['price'].apply(extract_billion).astype(np.float64)
<ipython-input-14-cf1588ec92cd>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [16]:
with open('data-to-db.json', 'w', encoding='utf-8') as file:
    json.dump(js, file, ensure_ascii=False)

In [24]:
# read old data and concat new data to total data
old_df = pd.read_csv('old-data.csv')
total_data = pd.concat([old_df, df], axis=0).reset_index()

In [25]:
# replace len(investor) < 30 to 'other'
mask_investor = total_data.investor.map(total_data.investor.value_counts()) < 30
total_data.investor =  total_data.investor.mask(mask_investor, 'other')
total_data.investor = total_data.investor.replace(np.nan, 'other')

In [35]:
# drop record has address is 'Thạch Thất' or 'Mê Linh'
total_data = total_data.drop(total_data.index[total_data['address'].isin(['Thạch Thất', 'Mê Linh'])])

# total_data[total_data['address'] == 'Mê Linh']

In [37]:
total_data

,index,address,bedroom,toilet,investor,acreage,price,lat,long,distance
0,0,Nam Từ Liêm,3.0,2.0,other,94.0,30.000000,21.012432,105.767395,9.030936
1,1,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,72.0,38.888889,21.019798,105.780708,7.527950
2,2,Cầu Giấy,2.0,2.0,tân hoàng minh group,82.0,34.146341,21.005806,105.794373,6.563944
3,3,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,69.0,29.000000,21.006529,105.753555,10.578458
4,4,Tây Hồ,4.0,2.0,tổng công ty đầu tư phát triển hạ tầng đô thị ...,148.0,35.810811,21.063967,105.805504,6.251858
...,...,...,...,...,...,...,...,...,...,...
29127,16395,Hoàng Mai,2.0,2.0,other,71.0,27.500000,20.956589,105.844887,8.087054
29128,16396,Hai Bà Trưng,2.0,2.0,tập đoàn vingroup,78.0,44.900000,20.996578,105.865196,3.831080
29129,16397,Long Biên,2.0,2.0,công ty cp đầu tư xây dựng số 9 hà nội,54.0,NaN,21.035978,105.906296,5.644863
29130,16398,Long Biên,2.0,NaN,tổng công ty đầu tư phát triển nhà và đô thị hud,85.0,30.600000,21.062691,105.908333,6.914078


In [38]:
total_data = total_data.fillna(total_data.median())

In [39]:
total_data

,index,address,bedroom,toilet,investor,acreage,price,lat,long,distance
0,0,Nam Từ Liêm,3.0,2.0,other,94.0,30.000000,21.012432,105.767395,9.030936
1,1,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,72.0,38.888889,21.019798,105.780708,7.527950
2,2,Cầu Giấy,2.0,2.0,tân hoàng minh group,82.0,34.146341,21.005806,105.794373,6.563944
3,3,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,69.0,29.000000,21.006529,105.753555,10.578458
4,4,Tây Hồ,4.0,2.0,tổng công ty đầu tư phát triển hạ tầng đô thị ...,148.0,35.810811,21.063967,105.805504,6.251858
...,...,...,...,...,...,...,...,...,...,...
29127,16395,Hoàng Mai,2.0,2.0,other,71.0,27.500000,20.956589,105.844887,8.087054
29128,16396,Hai Bà Trưng,2.0,2.0,tập đoàn vingroup,78.0,44.900000,20.996578,105.865196,3.831080
29129,16397,Long Biên,2.0,2.0,công ty cp đầu tư xây dựng số 9 hà nội,54.0,29.200000,21.035978,105.906296,5.644863
29130,16398,Long Biên,2.0,2.0,tổng công ty đầu tư phát triển nhà và đô thị hud,85.0,30.600000,21.062691,105.908333,6.914078


In [45]:
total_data = total_data.drop(['index'], axis=1)

In [46]:
total_data

,address,bedroom,toilet,investor,acreage,price,lat,long,distance
0,Nam Từ Liêm,3.0,2.0,other,94.0,30.000000,21.012432,105.767395,9.030936
1,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,72.0,38.888889,21.019798,105.780708,7.527950
2,Cầu Giấy,2.0,2.0,tân hoàng minh group,82.0,34.146341,21.005806,105.794373,6.563944
3,Nam Từ Liêm,2.0,2.0,tập đoàn vingroup,69.0,29.000000,21.006529,105.753555,10.578458
4,Tây Hồ,4.0,2.0,tổng công ty đầu tư phát triển hạ tầng đô thị ...,148.0,35.810811,21.063967,105.805504,6.251858
...,...,...,...,...,...,...,...,...,...
29127,Hoàng Mai,2.0,2.0,other,71.0,27.500000,20.956589,105.844887,8.087054
29128,Hai Bà Trưng,2.0,2.0,tập đoàn vingroup,78.0,44.900000,20.996578,105.865196,3.831080
29129,Long Biên,2.0,2.0,công ty cp đầu tư xây dựng số 9 hà nội,54.0,29.200000,21.035978,105.906296,5.644863
29130,Long Biên,2.0,2.0,tổng công ty đầu tư phát triển nhà và đô thị hud,85.0,30.600000,21.062691,105.908333,6.914078


In [47]:
total_data.to_csv('total-data.csv', index=False)